In [45]:
import os
import sys

import tensorflow as tf

from tensorflow.python.framework import graph_util
from tensorflow.python.tools import optimize_for_inference_lib

from keras.models import load_model, Model
from keras import backend

from tensorflow.core.framework import attr_value_pb2
from tensorflow.core.framework import graph_pb2
from tensorflow.python.framework import dtypes
from tensorflow.python.framework import tensor_util

# Inspect and extract graph
version = 3

model_path = os.path.dirname(os.path.realpath(sys.argv[1])) + '/bin/v' + str(version) + '/model.h5'

session = tf.Session()
backend.set_session(session)
backend.get_session().run(tf.global_variables_initializer())

model = load_model(model_path)
gd = session.graph.as_graph_def()

input_node_map = {}
for node in gd.node:
    if node.name not in input_node_map.keys():
        input_node_map[node.name] = node
    else:
        raise ValueError("Duplicate node names detected for ", node.name)

nodes_with_switch_op = [x for x in gd.node if x.op.lower().find('switch') != -1]
for n in [x for x in gd.node]:
    ints = [i for i in n.input]
    endswith1 = [ii for ii in ints if ii.endswith(':1')] #and 'Switch' in ii]
    if len(endswith1) > 0:
        for index, inn in enumerate(n.input):
            if inn in endswith1:
                new_input = inn[:-2]
                n.input.remove(inn)
                n.input.insert(index, new_input)

for n in nodes_with_switch_op:
    n.op = 'Identity'
    n.input.pop()

# input_node_map['dropout_1/cond/Switch_1']
# input_node_map['dropout_1/cond/pred_id']

# nodes_with_dropout = [x for x in gd.node if x.name.lower().find('dropout') != -1]
# nodes_with_dropout = [(n.name, [i for i in n.input]) for n in gd.node if n.name.lower().find('dropout') != -1]
# print(nodes_with_dropout)

# for n in gd.node:
#     if 'dropout_1/cond/Switch_1' in [i for i in n.input]:
#         print(n)

nodes_with_learningphase = [x for x in gd.node if x.name.lower().find('keras_learning_phase') != -1]
# print(nodes_with_learningphase)

for n in nodes_with_learningphase:
    n.op = 'Const'
    n.attr.get_or_create('value').CopyFrom(attr_value_pb2.AttrValue(tensor=tensor_util.make_tensor_proto(
            [False], dtypes.bool, [1])))
    del n.attr['shape']

output_graph_def = graph_util.convert_variables_to_constants(session, gd, ['main_output/Softmax'])

# optimized_graph_def = optimize_for_inference_lib.optimize_for_inference(
#         output_graph_def,
#         ["main_input"], # an array of the input node(s)
#         ["main_output/Softmax"], # an array of output nodes
#         tf.float32.as_datatype_enum)

# WRITE TO GRAPH
tf.train.write_graph(output_graph_def, ".", "inference_graph.pb", as_text=False)


INFO:tensorflow:Froze 8 variables.
Converted 8 variables to const ops.


'./inference_graph.pb'

In [23]:
def display_nodes(nodes):
    for i, node in enumerate(nodes):
        print('%d %s %s' % (i, node.name, node.op))
        [print(u'└─── %d ─ %s' % (i, n)) for i, n in enumerate(node.input)]